## Data Analyizer for cs584 course project
Auther: kun wu

## 1. Config keras to use GPU

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# import tensorflow as tf
# import keras

# config = tf.ConfigProto( device_count = {'GPU': 5} ) 
# sess = tf.Session(config=config)
# keras.backend.set_session(sess)

## 2. Load and Preprocess

### Load Glove pre-train embedding

In [2]:
import os

BASE_DIR = '/home/kwu14/data/cs584_course_project'

### Load data

In [3]:
import pandas as pd

train_df = pd.read_csv(os.path.join(BASE_DIR, 'preprocessed_train.csv'))

print(f'Data size: {train_df.shape}')

train_df.head()

Data size: (1804874, 46)


,Unnamed: 0,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,0,59848,0,cool like would want mother read realli great ...,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,1,59849,0,thank would make life lot less anxietyinduc ke...,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,2,59852,0,urgent design problem kudo take impress,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,3,59855,0,someth ill abl instal site releas,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,4,59856,1,haha guy bunch loser,0.021277,0.0,0.021277,0.87234,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


### Preprocess

In [4]:
train_df['target'] = train_df['target'].apply(lambda x: 1 if x > 0.5 else 0)
x_train = train_df['comment_text'].astype(str).values
y_train = train_df['target'].values

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)

vocab_size = len(tokenizer.word_counts)
max_len = 0
for seq in x_train:
    max_len = len(seq) if len(seq) > max_len else max_len
    
x_train = pad_sequences(x_train, maxlen=max_len)

print(f'Vocab size: {vocab_size}')
print(f'Max length: {max_len}')

Using TensorFlow backend.
/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kwu14/anaconda

Vocab size: 392342
Max length: 305


In [6]:
from glovevectorizer import load_glove_weights, generate_weights

glove_weights = load_glove_weights(os.path.join(BASE_DIR, 'glove.6B.300d.txt'))
weights = generate_weights(glove_weights, tokenizer.word_index)
print(f'weights shape: {weights.shape}')

weights shape: (392343, 300)


In [7]:
from keras.layers import Input, LSTM, Bidirectional, Embedding, Dense
from keras.models import Model

vocab_size, embedding_size = weights.shape
hidden_size = 100

words = Input(shape=(None, ), name='input_layer')
x = Embedding(vocab_size, embedding_size, weights=[weights], trainable=False, name='embedding_layer')(words)
x = Bidirectional(LSTM(hidden_size, return_sequences=False, name='lstm1'))(x)
# x = Bidirectional(LSTM(hidden_size, return_sequences=False), name='lstm2')(x)
x = Dense(100, activation='relu')(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=words, output=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, None)              0         
_________________________________________________________________
embedding_layer (Embedding)  (None, None, 300)         117702900 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 118,043,901
Trainable params: 341,001
Non-trainable params: 117,702,900
_________________________________________________________________


/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [ ]:
batch_size = 1024
epochs = 2

history = model.fit(
    x_train, y_train, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split=0.2
)

Train on 1443899 samples, validate on 360975 samples
Epoch 1/2
1241088/1443899 [========================>.....] - ETA: 3:58 - loss: 0.1406 - acc: 0.9552

In [ ]:
model.save_weights('rnn.model.h5')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

losses = history.history['loss']
epochs = range(len(losses))

plt.plot(epochs, losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()